<img src="./images/cmiyc2024_test.webp" alt="A kitten with a tophat at a computer. Aka JtR + H" width="800"/>

# Cracking CMIYC 2024 Test Hashes

**GETTING READY FOR CMIYC 2024!!!!**

### Overview

The goal of this notebook is to make it easy as possible for you to get a team registered and ready to participate in CMIYC. I'm hopefull I'll be updating it a lot running up to the contest, so I'd recommend using this as an example, but creating your own notebook to manage your teams cracking sessions.

### Major Features and Training Goals

1. How to register a team with CMIYC
2. TBD. Still need to work on this --Lakiw



### Creating a PGP Key (If Needed)

Let's face it. PGP is a pain, and you don't want to have to try and figure it out during the contest. But all communicaiton with the organizers (KoreLogic) need to be done via PGP encrypted messages. This includes creating a team and submitting hashes. To make this easier, I created some functions to help using the Python3 **pgpy** library.

The first function is **generate_read_pgp_private_key**. This will create a new PGP key (if one does not already exit) and write it to the key_file name that you send to it. If a PGP key already exists it'll validate the key and then return it. This way you aren't creating a new key every time this is run.

In [1]:
from lib_framework.pgp_mgr import generate_read_pgp_private_key

import os
import sys

key_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'pgp_key.asc'
    )

my_private_key = generate_read_pgp_private_key(key_file, email_address="YOUR_EMAIL_ADDRESS", team_name="YOUR_TEAM_NAME")

### Creating a Registration Email

To make it easier to encrypt/decrypt messages I created the **PGP_Mgr** class. I figured a class would be better than having individual functions since I didn't want to have to keep passing in my private key as well as KoreLogic's public key.

While I could try to integrate this with individual e-mail services, that seems like a lot of work so PGP_Mgr will output encrypted messages to this workbook. It's up to you then to actually copy/paste that message into an e-mail and send it to KoreLogic at sub-2024@contest.korelogic.com

Note, the register_team function will both print the message to your screen as well as return the message. Given the way JupyterLab displays newlines in returned messages I'd recommend hiding the return value using the **";"** at the end of the function call, and copy/pasting the printed version.

In [2]:
from lib_framework.pgp_mgr import PGPMgr, read_pgp_key

# Load up KoreLogic's submisison public key
korelogic_sub_public_key_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'defcon-contest-sub-2024.pub.asc'
    )
korelogic_sub_public_key = read_pgp_key(korelogic_sub_public_key_file)

default_attachment = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'submission.asc'
    )

# Create the PGPMgr class
pgp_mgr = PGPMgr(my_private_key, korelogic_sub_public_key, default_attachment)

# Create Submission e-mail
#pgp_mgr.register_team("Someone Didn't Update Their Teamname in the JupyterLab Framework by Lakiw");
pgp_mgr.register_team("YOUR_TEAM_NAME");

-----BEGIN PGP MESSAGE-----

wcBMA3EnRIyYyGY/AQf/baw++S7jPaXGjSLjEyYBADtDCKFjDYLXaxtqfOhghqb6
r7xysMatGAE6wPkgZngGHV3lXdG+t74MLG/PkPngZJip5XE+y/4ifq5mXFkauyni
i1dR2G2eLKzKaZtQjdnGvNmyC2akgXiEmGCjr3PoXCOK3IBpnNvUjiLrEI+vytBX
X/AWA0YJ6JTeLhKsvnd9xsWRPSCJxxB2oTHBAvDVR2tTJcQNCEGVijyUR2e3mJ+q
fzvLBya5ltU3teCmE8QQL6v7CNJBfwgXGp1B8/OVraUnZdR1KQslZYF+BnON1Pen
v/STILiBZEEkJY3zqLfbB+Wmby44HCE5j2CzT7HMOdLHWQH7BUT6NQLUyWDdXuHK
zmnYu9xoDbGxjYhCJHKzLZsk0s9kaO6cOA1El/C6ezqCxgKRsz7d9mL8Nt+V++5S
akQRoort7GgHkQ27w7hGxhgc6s5RPJn1nCLLrZub39RBpPmAvxd+RVoigEDC5yVH
fSSOgJAsT4y+XbS9zEpMmDLOO+KbU9vOHXANvKl3dVXAEoNnar5hZuhBuHEZlZcJ
ArJkpBO3Lo1oEubQTlHOmW1Bn/Ehng8GI5ymrWp+FkK8IZZeTR/vpjgUD6OEDYxs
SoBuKj9v0bduSDgSVHxYE84C1w8GAqlDsXQIOhDyu9XqlU9LSBd+zosY8F8u8dKY
0JUQ7ztW696tagKcIgKmLsw7dCWjoinWfJ1RQ3XxlA6VgUXUl7RUUtH4bI70xN75
gdvcA61UGkawd2a5Die2OGs/R7ifiOkllHTXTWqDn0t9sHaVOM46EouSihZmJob/
okEp+lOFhnXiyHF+HpA39he+bPGvD9lCv7aW2Ly/IlN4E92Ihfjd2dDsMJI//ItF
PpXy50DPP1pIkUYTV61yqdJDqVx2RQYCKJM2kxtRm/9QcGODDG76/brZ14AWz

In [3]:
# Now we need to validate the confirmation e-mail
default_attachment = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'korelogic_comms.asc'
    )
challenge_response = pgp_mgr.decrypt_msg(default_attachment)

Thank you, your key has been successfully confirmed.

You can start submitting cracks as soon as the contest has started.

Keep an eye on the contest website; we may release some test hashes
a few days before the contest, so teams can confirm their submission
process works.

Don't forget to let us know if you want to compete as a 'Pro' team.




In [4]:
pgp_mgr.encrypt_msg(challenge_response);

-----BEGIN PGP MESSAGE-----

wcBMA3EnRIyYyGY/AQf+KOXG+Oxc9G00sy0ZzgfBPcMwW7Wgc1tQXi5A0+Sp0j33
8NMAP1caXmVFGUPvD0zXFuyCf8Fdqwew4iSlmO3NE9mSXPeQiRYQcTF5JF2kcirf
DjPqPOM7OOZobJWH2kUJnN0kLcSOWCXDMWZSNNmelYoPautOY2XIKOPONBWj+KiO
CCTy412ta2GPY/iYe88Kl+fFwFe9DTPFfKbpLjoKR0lg5IThgxMmK1fJ2oTiDdI8
pQ34JLfdqQ4++nRr8X3MZaIRo8CR6cKua1EOGV5jzGuZ/2X6SlbfF5Z1E7ncbAXC
qFY8VN3PJM1Te2+hTyBd+Xx8O56eAwI3wFx/YIBNJtLC1gFZCk0it7GcVZE6czbp
Oovcde5pSAVzv5PIc92C6ebBGIr6tYIof1T0pS9KmmTnVAmxDqHVgmgW4EPjyYp2
C0hrJFQFNGCbxhuSquPM4af5BwsirDhtl912qqIN5tn0hXhnReiX75J3TYM/WHqO
DFnU+bPSkzqCX1+RieVmpb51ga/e8H/QTCfcL6DO1DvAimmUJZ03u5AmRxUu18BE
w/UJd+YfrDhkJBrKC81H+EUE9QvGD28NjxDcnnRFCgxlw+Gwjj17OAUGQ2chEIva
R44tGUZI7rGB+320b8M3OsYXIbGN9ebu2hw21KZCbEKzH9Nbcu0p0ptrTq6/5Fqg
XC8K3k6TZvkwFfZN76XoRDIdvSGh0Va000laKBeCz30r8JKd/W/CoUY/0kdJa91A
03szpQjG9stkGUlJXuUshNgXjPiWDjdJbIL2ZnY/kI+cfxuWOI0fI418CoFSxPJw
7jo2YFF4/I/ZKuW8MEERAUluAp3BZz5AZC4iWmWpTGFG5fnroWQeBwumoZ52gH6x
nUkNgEEEvWRQtYF6bvwvC8VE/pJ35bNji9xTcOhFmQ4zdtwFjiX9toybJdarl